In [1]:
import pandas as pd
import os

Get text file

In [2]:

swed_file = open(os.getcwd()+"\\dat410_europarl\\europarl-v7.sv-en.lc.sv", mode='rt', encoding='utf-8')
en_file = open(os.getcwd()+"\\dat410_europarl\\europarl-v7.sv-en.lc.en", mode='rt', encoding='utf-8')

# read all texts
swed_text = swed_file.read()
en_text = en_file.read()
# close the file
swed_file.close()
en_file.close()

In [3]:
swed_sentences = swed_text.strip().replace("&quot;","").split('\n')
en_sentences = en_text.strip().replace("&quot;","").split('\n')


In [4]:
swed_sentences[7]

'skulle det vara möjligt för er , fru talman , att skriva ett brev till den srilankesiska presidenten i vilket parlamentets beklagande uttrycks över hans och de övriga brutala dödsfallen i sri lanka och uppmanar henne att göra allt som står i hennes makt för att få en fredlig lösning på en mycket komplicerad situation ?'

# Warmup

In [5]:
from collections import Counter, defaultdict

The 10 most frequent words in Swedish

In [6]:
swed_clean_text = swed_text.replace(".","").replace(",","").replace("&quot;","").replace("\n","").replace("  ","").strip()
swed_words = swed_clean_text.split(' ')
swed_counter_Words = Counter(swed_words)
swed_counter_Words.most_common(10)

[('att', 8896),
 ('och', 6110),
 ('i', 5715),
 ('det', 5347),
 ('för', 4632),
 ('som', 4425),
 ('av', 3961),
 ('är', 3680),
 ('en', 3570),
 ('vi', 3096)]

The 10 most frequent words in english

In [7]:
en_swed_text = en_text.replace(".","").replace(",","").replace("&quot;","").replace("\n","").replace("  ","").strip()
en_words = en_swed_text.split(' ')
en_counter_Words = Counter(en_words)
en_counter_Words.most_common(10)

[('the', 18235),
 ('of', 9119),
 ('to', 8514),
 ('and', 5910),
 ('in', 5497),
 ('is', 4139),
 ('a', 4083),
 ('that', 3920),
 ('this', 2924),
 ('for', 2657)]

In [8]:
def get_probability(word,counter_Words):
    value_word = counter_Words.get(word)
    value_word= value_word if value_word is not None else 0
    total_Count=sum(counter_Words.values())
    return value_word/float(total_Count) * 100


what is the probability that it is speaker?

In [9]:
get_probability('speaker',en_counter_Words)

0.0032880405744206884

What is the probability that it is zebra?

In [10]:
get_probability('zebra',en_counter_Words)

0.0

In [11]:
en_sentences[0]

'i declare resumed the session of the european parliament adjourned on friday 17 december 1999 , and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period .'

In [12]:
en_sentences[0]

'i declare resumed the session of the european parliament adjourned on friday 17 december 1999 , and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period .'

#  Language modeling

## bigram
1. select pairs

In [13]:
bigram_model = defaultdict(lambda: defaultdict(lambda: 0))
for sentance in en_sentences:
    words=sentance.split(' ')
    len_words=len(words)
    for i in range(0,len_words-3):
        w1,w2 = words[i:i+2]
        bigram_model[w1][w2] += 1

2. calculate probability

In [14]:
for w1 in bigram_model:
    total = sum(bigram_model[w1].values())
    for w2 in bigram_model[w1]:
        bigram_model[w1][w2] /= total

predict the next word

In [15]:
dict(bigram_model['declare'])

{'resumed': 0.5,
 'an': 0.16666666666666666,
 'that': 0.16666666666666666,
 'our': 0.16666666666666666}

In [16]:
dict(bigram_model['secular'])

{'arm': 0.5, 'millennium': 0.5}

In [17]:
pairs  =bigram_model.items()
pairs

dict_items([('i', defaultdict(<function <lambda>.<locals>.<lambda> at 0x000002B36484A8B0>, {'declare': 0.001015572105619499, 'would': 0.16926201760324983, 'should': 0.04197698036560596, 'feel': 0.008463100880162491, 'shall': 0.02098849018280298, 'will': 0.03148273527420447, 'believe': 0.045023696682464455, 'want': 0.020311442112389978, 'can': 0.02979011509817197, 'have': 0.07244414353419093, 'had': 0.004739336492890996, 'hope': 0.02911306702775897, 'am': 0.11374407582938388, 'admit': 0.0006770480704129993, 'propose': 0.003046716316858497, 'was': 0.011509817197020988, 'do': 0.03249830737982397, 'even': 0.00033852403520649965, 'therefore': 0.012525389302640487, 'indicated': 0.00033852403520649965, 'might': 0.0016926201760324984, 'regret': 0.003723764387271496, 'did': 0.003723764387271496, 'think': 0.042315504400812456, 'cannot': 0.00914014895057549, 'accept': 0.0006770480704129993, 'welcome': 0.009478672985781991, 'still': 0.0016926201760324984, 'congratulate': 0.002031144211238998, 'com

Find long sentaance

In [18]:
long_sentances = list(filter(lambda sentance: len(sentance.split(' ')) > 100, en_sentences))

probability of the whole sequence

In [19]:
def probability_sentances(sentances):
    probability_sentances_array=[]
    for sentance in sentances:
        words=sentance.split(' ')
        len_words=len(words)
        multiple_probability = 1
        for i in range(0,len_words-3):
            w1,w2 = words[i:i+2]
            multiple_probability *= bigram_model[w1][w2]
        probability_sentances_array.append((len_words,multiple_probability))
    return probability_sentances_array

In [20]:
probability_sentances(long_sentances)

[(103, 3.9804137166732987e-169),
 (115, 1.8380791576565445e-185),
 (110, 1.893893136951211e-169),
 (118, 5.055905815628088e-188),
 (128, 9.559051777437025e-196),
 (105, 9.234294300990775e-166),
 (122, 1.8243097991719043e-187),
 (147, 2.0724393803976357e-232),
 (112, 3.1153328115092984e-185),
 (107, 2.5353977942196483e-154),
 (113, 1.3521808972251636e-182),
 (125, 7.378233577445764e-200),
 (141, 6.112009881848214e-242),
 (102, 8.591268609363214e-157),
 (113, 6.271327686822779e-182),
 (118, 3.5595415445903497e-187),
 (118, 8.539575838803183e-191),
 (124, 1.2238288748765904e-174),
 (131, 2.953075712105567e-202),
 (121, 4.7905508686311034e-178),
 (185, 3.383482851728785e-288),
 (129, 3.728036288794875e-202),
 (110, 2.567260330987985e-173),
 (113, 6.252598270242032e-177),
 (113, 2.0045496101195e-183)]

In [21]:
short_sentances = list(filter(lambda sentance: len(sentance.split(' ')) < 20, en_sentences))

In [115]:
probability_sentances(short_sentances)

[(12, 1.9363885817087248e-13),
 (14, 1.1146276632180963e-18),
 (11, 1.7194854567000422e-12),
 (19, 1.6134127480583728e-28),
 (14, 3.9383828016894384e-18),
 (14, 5.827948176960894e-17),
 (12, 5.737137943398884e-13),
 (16, 1.0576779909809933e-21),
 (8, 3.3958111799210166e-09),
 (13, 3.543786540880243e-15),
 (18, 9.064330208095826e-23),
 (13, 1.8206208866225058e-17),
 (19, 1.7918476303140873e-25),
 (14, 2.4922038750757835e-16),
 (10, 1.7653209207513511e-09),
 (15, 7.515408990319437e-21),
 (17, 9.601028727703009e-26),
 (11, 2.5768000388569592e-14),
 (4, 0.975609756097561),
 (17, 2.6615735964889454e-25),
 (14, 3.493186837832068e-16),
 (7, 1.0748263681127036e-07),
 (14, 6.822180195683467e-18),
 (17, 5.057446716398626e-25),
 (12, 6.103978765279483e-18),
 (15, 4.6900923449167446e-18),
 (6, 0.0004678894319587283),
 (11, 7.536204869630549e-13),
 (18, 2.7109100084655502e-17),
 (18, 3.0315067133344304e-28),
 (15, 2.65507438378622e-13),
 (15, 7.14348148458201e-18),
 (8, 6.13292373008249e-11),
 (5, 